<a href="https://colab.research.google.com/github/BhattacharyaSumit/Style-Transfer-using-Pytorch/blob/master/Sentiment_Analysis_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt"

--2019-05-17 14:10:21--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M   128MB/s    in 0.3s    

2019-05-17 14:10:22 (128 MB/s) - ‘reviews.txt’ saved [33678267/33678267]



In [2]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt"

--2019-05-17 14:10:27--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2019-05-17 14:10:27 (4.41 MB/s) - ‘labels.txt’ saved [225000/225000]



In [0]:
import numpy as np

# read data from text files
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

In [5]:
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [6]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [0]:
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [0]:
[c for c in reviews_split[:1]]

['bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   ']

In [0]:
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
sort = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word : i for i , word in enumerate(sort,1)}
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints =[]

In [0]:
for c in reviews_split:
  reviews_ints.append([vocab_to_int[p] for p in c.split()])

In [10]:
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [0]:
labels = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels])

In [12]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [14]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
new = []
for i in range(len(reviews_ints)):
  if len(reviews_ints[i]) != 0:
    new.append(encoded_labels[i])
    
new_ = []
for i in range(len(reviews_ints)):
  if len(reviews_ints[i]) != 0:
    new_.append(reviews_ints[i])

reviews_ints = np.array(new_)
encoded_labels = np.array(new)

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function
    trunc_pad  = [review_int[:seq_length] + [0]*(seq_length - len(review_int) ) for review_int in reviews_ints]
    features= np.array(trunc_pad)
    
    return features

In [0]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length , "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
#print(features[:30,:10])

In [0]:
split_frac = 0.8

train_x, train_y = features[0:int(split_frac*len(features))], encoded_labels[0:int(split_frac*len(features))]

validation_x , validation_y = features[int(split_frac*len(features)): int((split_frac/2 + 0.5)*len(features))], encoded_labels[int(split_frac*len(features)): int((split_frac/2 + 0.5)*len(features))]

test_x, test_y = features[int((split_frac/2 + 0.5)*len(features)) : len(features)], encoded_labels[int((split_frac/2 + 0.5)*len(features)) : len(features)]



In [18]:
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(validation_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(validation_x), torch.from_numpy(validation_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [20]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[  697,    26,   601,  ...,     0,     0,     0],
        [  596,  1021,     2,  ...,     9,    22,     1],
        [   11,     6,     3,  ...,   367,   976,   137],
        ...,
        [   84,     4,    31,  ...,    86,  1734,  6037],
        [  101,   148, 28156,  ...,   229,     2,    43],
        [   24,    65,     3,  ...,    43,     3,    50]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 1])


In [21]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)

        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch = x.size(0)
        
        emb = self.embedding(x)
        lst, hidden = self.lstm(emb, hidden)
        
        lstm_out = lst.contiguous().view(-1, self.hidden_dim)
        
        drop = self.dropout(lstm_out)
        fc = self.fc(drop)
        sig_out = self.sig(fc)
        
        sig_out = sig_out.view(batch, -1)
        sig_out = sig_out[:, -1]
        
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden
        

In [37]:
vocab_size = len(vocab_to_int) + 1
output_size = 1
embedding_dim =  400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
lr=0.001

criterion = nn.BCELoss() #Binary Cross Entropy Loss
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [39]:
epochs = 4 

counter = 0
print_every = 100
clip=5 # gradient clipping


if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())
                
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))   

Epoch: 1/4... Step: 100... Loss: 0.684939... Val Loss: 0.691097
Epoch: 1/4... Step: 200... Loss: 0.704176... Val Loss: 0.688539
Epoch: 1/4... Step: 300... Loss: 0.722777... Val Loss: 0.695676
Epoch: 1/4... Step: 400... Loss: 0.694913... Val Loss: 0.693226
Epoch: 2/4... Step: 500... Loss: 0.686571... Val Loss: 0.693367
Epoch: 2/4... Step: 600... Loss: 0.698286... Val Loss: 0.693130
Epoch: 2/4... Step: 700... Loss: 0.684000... Val Loss: 0.694015
Epoch: 2/4... Step: 800... Loss: 0.693794... Val Loss: 0.693180
Epoch: 3/4... Step: 900... Loss: 0.694487... Val Loss: 0.693192
Epoch: 3/4... Step: 1000... Loss: 0.696971... Val Loss: 0.693501
Epoch: 3/4... Step: 1100... Loss: 0.690748... Val Loss: 0.693177
Epoch: 3/4... Step: 1200... Loss: 0.690079... Val Loss: 0.693185
Epoch: 4/4... Step: 1300... Loss: 0.687854... Val Loss: 0.693113
Epoch: 4/4... Step: 1400... Loss: 0.683692... Val Loss: 0.693522
Epoch: 4/4... Step: 1500... Loss: 0.686847... Val Loss: 0.693292
Epoch: 4/4... Step: 1600... Loss: 

In [41]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.692
Test accuracy: 0.514


In [0]:
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.' 

test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

In [0]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    reviews_infer = test_review.lower() # lowercase, standardize
    all_text_infer = ''.join([c for c in reviews_infer if c not in punctuation])
    reviews_split_infer = all_text_infer.split('\n')
    all_text_infer = ' '.join(reviews_split)
    words_infer = all_text_infer.split()
    
    counts_infer = Counter(words_infer)
    vocab_infer = sorted(counts, key=counts_infer.get, reverse=True)
    vocab_to_int_infer = {word: ii for ii, word in enumerate(vocab_infer, 1)}

    
    reviews_ints_infer = []
    for review in reviews_split_infer:
        reviews_ints_infer.append([vocab_to_int_infer[word] for word in review.split()])
        
    features_infer = pad_features(reviews_ints_infer, sequence_length)
    
    feature_tensor = torch.from_numpy(features_infer)
    
    h = net.init_hidden(1)

    net.eval()
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
        

    output, h = net(feature_tensor, h)
    pred = torch.round(output.squeeze())
    
    # print custom response based on whether test_review is pos/neg
    sent = 'Positive' if pred==1 else 'Negative'
    
    print("\t{}".format(sent))

In [87]:
seq_length=200
predict(net, test_review_pos, seq_length)

	Positive


In [88]:
seq_length=50
predict(net, test_review_neg, seq_length)

	Negative
